# Quick low effort AutoML Solution using FLAML
For this minimal solution, I entered the data as is into FLAML and let the library run different models for 20 minutes to find a solid solution. This allows a quick submission which is close to best submissions and requires nearly no effort.

In [ ]:
pip install flaml

In [ ]:
def reduce_dtypes(df, verbose=True):
    """
    reduces the datatype of numerical columns to smallest possible. 
    dtypes: "int8", "int16", "int32", "int64", "float16", "float32", "float64"
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    int_types = [np.int8, np.int16, np.int32, np.int64]
    float_types = [np.float16, np.float32, np.float64]
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                for int_type in int_types:
                    if c_min > np.iinfo(int_type).min and c_max < np.iinfo(int_type).max:
                        df[col] = df[col].astype(int_type)
                        break
            else:
                for float_type in float_types:
                    if c_min > np.finfo(float_type).min and c_max < np.finfo(float_type).max:
                        df[col] = df[col].astype(float_type)
                        break
    if verbose:
        end_mem = df.memory_usage().sum() / 1024 ** 2
        print(f"Mem. usage decreased to {end_mem:.2f} Mb ({100*(start_mem - end_mem)/start_mem:.1f}% reduction)")
    return df

## Load the data

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv', index_col='id')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv', index_col='id')

reduce_dtypes(train)
reduce_dtypes(test)

X_train, y_train = train.drop('target', axis=1), train[['target']].values.flatten()
X_test = test

In [ ]:
X_train.head()

# Fit the Model

In [ ]:
from flaml import AutoML

automl = AutoML()

automl_settings = {
    "time_budget": 60 * 60, # 60 min
    "metric": 'roc_auc',
    "task": 'classification',
    "log_file_name": "nov.log",
    "estimator_list": ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree'],
    "ensemble": True,
    "seed": 1337,
    "n_jobs": 1, # stacking crashes with -1
}

automl.fit(X_train=X_train, y_train=y_train, **automl_settings)

## Predict

In [ ]:
preds = automl.predict_proba(X_test)

submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
submission["target"] = preds[:, -1]
submission.to_csv("submission.csv", index=False)